In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2022-06-25 18:17:16--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2022-06-25 18:17:17--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3bceb72b1a1159c7299fce43be.dl.dropboxusercontent.com/cd/0/inline/Bn4xSemCOLwNWT2rOWEU45gNZP0ztVC7uH_Uc54D2UnAuK1r130Vgqi-v2ekTb-Zn_rdRVSh3k3g1X9j3FJYLNZ4oVShlktpYHE8V7WAgTMWDBlT35KIC9RkNhqvXRolbBb3PzqqAyFUX1sA6KOZ9qZHSGDxgWtV6OfLIqKwFym2IQ/file# [following]
--2022-06-25 18:17:17--  https://uc3bceb72b1a1159c7299fce43be.dl.dropboxusercontent.com/cd/0/inline/Bn4xSemCOLwNWT2rOWEU45gNZP

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [3]:
# Sentiment Analysis

In [4]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout

In [5]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, '<br />', ' '), label)
  )

In [6]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'I saw the MST3K version of "Deathstalker III" and loved the movie so much -- even "unmystied" -- that I decided to watch the entire series of "Deathstalker" films. I bought I and II and settled down for a laugh.  Nothing about "Deathstalker I" was funny on any level and when the credits rolled I was embarrassed and regretful that I had bought it! Too much ugliness and nudity. I guess either "DS 3" was a much cleaner production or MST3K really edited a lot because I expected something similar, i.e. stupid and carefree and simple. I was wrong. Even at $6.99 it seemed a waste of money. I didn\'t even open "DS 2" as I will return it tomorrow. Now I\'ll probably just throw away this DVD as I can\'t return it and no one wants it -- including myself! So really, don\'t bother with this one. Even the nudity (lots of it, btw) is uninspiring and icky.'
0


In [7]:
model = Sequential()

In [8]:
model.add(Input(shape=(1,), dtype="string"))

In [9]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_len
)

In [10]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [11]:
model.add(Embedding(max_tokens + 1, 128))

model.add(GRU(64))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [12]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 18s 14ms/step - loss: 0.5863 - accuracy: 0.6716
Epoch 2/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4233 - accuracy: 0.8054
Epoch 3/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3874 - accuracy: 0.8225
Epoch 4/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3677 - accuracy: 0.8316
Epoch 5/10
782/782 [==============================] - 11s 13ms/step - loss: 0.3501 - accuracy: 0.8400
Epoch 6/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3319 - accuracy: 0.8469
Epoch 7/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3122 - accuracy: 0.8550
Epoch 8/10
782/782 [==============================] - 11s 14ms/step - loss: 0.2934 - accuracy: 0.8613
Epoch 9/10
782/782 [==============================] - 11s 13ms/step - loss: 0.2769 - accuracy: 0.8690
Epoch 10/10
782/782 [==============================] - 11s 14ms/step - loss: 0.252

In [14]:
model.evaluate(test_data)

782/782 [==============================] - 8s 10ms/step - loss: 0.7316 - accuracy: 0.7787


[0.7316035032272339, 0.7787200212478638]

In [15]:
text = "I loved the movie !"

In [16]:
model.predict([text])

array([[0.9951545]], dtype=float32)